In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|███████████████████████| 170498071/170498071 [00:02<00:00, 63931441.77it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [18]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(512 * 2 * 2, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 10)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = x.view(-1, 512 * 2 * 2)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.fc4(x)
        return x

In [19]:
net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(25):  # loop over the dataset multiple times

    running_loss = 0.0
    correct = 0
    total = 0

    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)  # get the predicted class labels
        total += labels.size(0)
        correct += (predicted == labels).sum().item()  # compare with the true labels

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'Epoch: {epoch + 1}, Batch: {i + 1}, Loss: {(running_loss / 2000):.3f}, Accuracy: {(correct / total) * 100:.2f}%')
            running_loss = 0.0

print('Finished Training')

Epoch: 1, Batch: 2000, Loss: 2.021, Accuracy: 21.48%
Epoch: 1, Batch: 4000, Loss: 1.739, Accuracy: 27.21%
Epoch: 1, Batch: 6000, Loss: 1.600, Accuracy: 31.71%
Epoch: 1, Batch: 8000, Loss: 1.489, Accuracy: 35.16%
Epoch: 1, Batch: 10000, Loss: 1.404, Accuracy: 38.02%
Epoch: 1, Batch: 12000, Loss: 1.357, Accuracy: 40.28%
Epoch: 2, Batch: 2000, Loss: 1.284, Accuracy: 54.95%
Epoch: 2, Batch: 4000, Loss: 1.232, Accuracy: 55.84%
Epoch: 2, Batch: 6000, Loss: 1.185, Accuracy: 56.75%
Epoch: 2, Batch: 8000, Loss: 1.157, Accuracy: 57.61%
Epoch: 2, Batch: 10000, Loss: 1.148, Accuracy: 58.28%
Epoch: 2, Batch: 12000, Loss: 1.136, Accuracy: 58.80%
Epoch: 3, Batch: 2000, Loss: 1.027, Accuracy: 65.29%
Epoch: 3, Batch: 4000, Loss: 1.025, Accuracy: 65.09%
Epoch: 3, Batch: 6000, Loss: 1.017, Accuracy: 65.21%
Epoch: 3, Batch: 8000, Loss: 1.010, Accuracy: 65.50%
Epoch: 3, Batch: 10000, Loss: 0.983, Accuracy: 65.78%
Epoch: 3, Batch: 12000, Loss: 0.985, Accuracy: 66.00%
Epoch: 4, Batch: 2000, Loss: 0.928, Accu

In [20]:
# Save the entire model
torch.save(net.state_dict(), './cifar10_95.pth')